## case 4. Real-Time Translation Learning to Decode

- Generation하는 part를 집중적으로 보자.


- 지금까지 설명한 것들을 보면, sentence representation부터 Autoregressive Model을 써서 sequence modeling(scorin)하는 지를 보았고, Training에서는 supervised learning과 같이 만들어 back propagatin, SGD, early stopping등등 많은 것을 보았지만, 빠진 것이 있다.


- training할 때 Source Sentence나 Target sentence(translation)이 주어졌을 때 scoring하는 방법은 배웠지만, Given sentence가 Target sentence(translation)은 안주어지고, source Sentence만 주어졌을 때 어떻게 모델이 선호하는 translation을 찾는지(?)에 대한 문제는 말하지 않았다.


- 이걸 지금까지 뒤로 미루어서 말하는 이유가 아직 어떻게 하는 것이 맞는 것인지 모른다. 다. 모두들 각자 어떠한 방식으로든 (ex) google translation에서 하루에 billion의 translation을 할 때도)하는데 이게 잘 하는건지 못하는 것인지 모르고 하는 것이다.



### Decoding from a recurrent language model

- 계속 말했듯이 Language Model이라는 것이 Scoring이고 Scoring을 할 수 있으면 Sentence Generation이 가능하므로 다음과 같이 search할 수 있다.


- ** Decoding - 0. Exhaustive Search (Brute- force search) **
    - Simple and exact decoding algorithm
    - Score each and every possible translation & Pick the best one
    - 가능한 경우를 모두 구해서 문제의 해결 방법을 찾는 것
        - 모든 가능한 translation sequence를 시도하여 각각에 대해 scoring해 결과가 가장 높은 score를 내보내는 것.
    - cons: 물리적으로 불가능
    - <img  src="./image/img_c4-0.PNG" width="50%">  

- ** Decoding - 1. Ancestral Sampling(Forward Sampling) ** (Forward Sampling의 [기계개론 설명](https://github.com/NamSahng/Summary/blob/master/Intro2AI%26ML/W10%20Sampling%20Based%20Inference%20part1.ipynb)):
    - Language Model이나 Autoregressive Model이라는 것이 generative story가 있는 것이다.(첫 단어 만들고, 그 만든 것으로 다음단어 만들고 그 두 단어 만든 것으로 세 단어 만들고 ...) 이것이 어떻게 보면 Probabilistic Graphical Model(directed graph)을 만드는 것이며 
        - 이 것이 나오면 순서를 따라가면서 매번 Conditional Distriution에서 sample을 하나 뽑을 수 있다.
    - Conditional Distriution의 분산이 작으면 다음에 올 단어가 몇 개 안되겠지만, 그렇진 않을 것이다. 
    - 이렇게 샘플링 다 해서 EOS가 나온 문장들을 보고 score가 높은 것을 Pick한다.
    - <img  src="./image/img_c4-1.PNG" width="60%"> 
        - One symbol at a time from $ \tilde{x}_{t} \sim x_{t} | x_{t-1}, ... , x_{1}, Y $  
        - Until $ \tilde{x}_{t} = \left\langle EOS \right\rangle $
    - 특징: 나름 Efficient, unbiased sampling 이다.
    - cons: 샘플을 많이 뽑아야하고, variance 가 높다.
    -  sentence space가 굉장히 exponential하게 커서, sampling을 엥간히 많이 하지 않는 이상 괜찮은 translation을 pick할 보장이 안된다. (Variance가 높다.)
        - 서비스를 할 때 굉장히 문제됨. (어제한 translation은 굉장히 좋았는데 오늘하니까 말도 안되는 현상 같은 문제)

    - 그래서 이런 High variance와 pretty inefficient한 문제를 해결하기 위해
    - ML에서 보통 사용하는 Bias-Variance tradeoff를 이용해 Bias를 늘려
    - 만들어 Variance를 낮춰 정확한 답은 안주지만 할 때마다 똑같은 답을 주는 방법을 할 수 있다..
        - cf) bias and variance trade-off (ref. 문일철교수님, "기계학습개론 1" & 앤드류 응, "DeeplearNG")
        - a. Variance: (우리가 가진 데이터셋과 모든 데이터 셋을 봤을 때의 차이에 생긴다) 
            - 해결방안: generalization과 관련. 이는 regularization(data aug, batch norm 등)으로 해결한다.

        - b. Bias: (데이터 셋에는 문제가 없다.) 우리가 가진 모델(Approximation)의 한계점.  
             - 해결방안: 다른 hypothesis(hyper-parameter 변경, 모델 변경(comlex) 등)를 사용.
        - 앤드류 응의 DeeplerNG에서는 초기 ML시대에는 trade off가 있었지만, 딥러닝에서는 정규화를 잘했고 네트워크와 데이터는 trade off 없이 잘 된다고 한다.(여기서 Language Generation에서는 최종적으로 PGM과 같은 ML방식으로 scoring해서 아직 trade off 존재(?))
            - <img  src="./image/img_c4-2.PNG" width="50%"> 


- ** Decoding - 2. Greedy Search **
    - Autoregressive 모델과 잘 맞아 떨어진다. 매번 단어 하나하나 고를 때, 지금까지 주어진 단어를 보고 (conditional) distribution을 보고 distribution상에서 가장 likely한 단어를 뽑는 것.
        - Pick the most likely symbol each time $ \tilde{x}_{t} = argmax_{x} log(p(x|x_{<t}, Y)) $  
        -  Until $ \tilde{x}_{t} = \left\langle EOS \right\rangle $
    - 말이 되는 것 같지만 굉장히 suboptimal하다.
        - 초반의 선택에 너무 dependent하여, 돌아와서 더 좋은 sequence를 고를 수가 없다.
    - 하지만 그래도 Pros: Super-efficient computation and memory
    - 실제로 하면, 말도 안되는 translation을 많이 한다.
    - <img  src="./image/img_c4-3.PNG" width="40%"> 


- ** Decoding - 3. – Beam Search **
    - Greedy Search는 하나만 했다면 가능한 여러 개(K 개) 후보를 같이 들고간다.
        - 가장 score가 좋았던, K개 (k를 Beam width라고도 함)
    - <img  src="./image/img_c4-4.PNG" width="80%">
    - K가 무한대면 exact search와 다를게 없다.
    - K를 늘린다고, 성능이 monotonic하게 증가하진 않음.
        - val loss를 보고, k를 결정하는 것이 최신의 standard 방법이다.
    - <img  src="./image/img_c4-5.PNG" width="60%">
    - Greedy와 Beam width(# of Chains) 5, 10 BLEU, Loss 비교
    - Greedy보다 확연히 score가 높긴하는데 <img  src="./image/img_c4-6.PNG" width="70%">



- 하지만 Decoding하는 것에서 안타까운 점은 얼마나 잘하는지가 모르고, 불문명하다. 
    - 모델이 진짜 잘 할 수 있는 것에서 얼마나 떨어져 있는 것인지, 
    - 얼마나 optimal하게 하고 있는 것인지
    - search를 더 잘하면 확실히 더 잘할 수 있는지
    - 그래서 search 알고리즘을 만들어 보면 잘될 때도 있고 안될 때도 있어 이 것을 하는 것이 맞나 싶을 때도 많다.
    - 그리고 고민하다 보면 이것을 안하는게 낫다는 결론이 나온다.
    
    
- training이 끝나고 decoding을 한다는 것은 모델의 입장에서 source sentence로부터 가장 likely한 translation sentence를 찾는 것인데, 이 것이 우리가 정말 원하는 것이냐 하면 살짝 다른 문제이다.
    - 즉 translation의 objective라는 것 자체가 conditional language model과 딱 떨어지진 않으며, 실제 application마다 objective는 달라질 것이다. 
        - ex) 실시간 번역 시스템의 objective: 품질(quality) 높게, 딜레이(delay)는 낮게
        - 어린이들을 위한 번역 시스템: 품질은 높여야 하지만 텍스트 난이도(text difficult)는 낮게
        - 온디바이스(on-device)에서의 번역시스템: 품질은 높게 계산복잡도(computational complexity)는 낮게
    - 따라서, decoding에서 그냥 score높은 sentence로 한다는 것은 quality외에는 나머지를 control안하겠다는 것이다.
    - 그래서 generate하는 것을 learning할 수 있는가.
        - 이를 위해 NN에 대해 다시 생각해보자.
        
        
        
### Learning to decode

- Neural Net 중 CNN을 생각해보면, Convolution-pooling을 output까지 엄청 여러 번한 다음에 Softmax로 Class conditional distribution으로 구분한다.
    - Convolution-pooling이라는 것은 최대한 input의 정보를 버리는 것이라고 볼 수 있다.
    - ex) 팬다에서 배경을 잊고, forground에서 팬다 같지 않은 나무를 잊고, 마지막에 팬다 같은 얼굴로 판단한다.
        - 그리고 중간단계의 정보를 보면 output의 정보보다 많은 정보와 잘 organized되어어있다. 버리기위해서는 organized되어야하니까.
    - <img  src="./image/img_c4-7.PNG" width="70%">
    - 이와 같이 RNN이나 Neural Autoregressive model에서도 매번 다음 토큰을 predict할 때 hidden activation을 보면 실제로 뽑아낸 token 자체보다 훨씬 정보가 많이 있다.
        - 어떤 토큰이 가능성이 있었고 없었고 왜 그런지 까지의 정보
    - 이 정보를 이용해 decoding을 하는 search해주는 알고리즘을 train할 수 있겠냐.
    - Computer vision은 보기가 좋아서 사람들도 많이 연구하여 Hidden layer가 무얼하는지 꽤 잘알 수있다.
    - 이에 비해 NLP 특히 discrete token에 대해서 아는 것이 별로 없다. (단어당 activation이 많이 됬는지 안됬는지 정도)
    - RNN 에서 은닉층 탐색을 해본 결과 NN 과 달리 봐도 시각적으로 어떤 특정 패턴이 있는지 구별할 수 없다.
        - 1000dimension을 low dimension으로 non-linear projection한 시각화와 hidden layer에서 1000dimension에서 시각화 (사람이 패턴을 찾을 수 없고 없는게 당연)
        - <img  src="./image/img_c4-8.PNG" width="80%">
    - 근데 우리는 pattern을 못찾아도 NN은 굉장히 잘 찾을 수 있다.
    - 그래서 우리가 직접 해석하는 것이 아니라, NN은 이것을 해석을 잘하니까
    - 또 다른 NN이 이것을 해석하도록 하자는 것이 아이디어이다. $ \to $ Trainable Decoding
    

### Trainable Decoding
    
- 지금까지 했던 것들을 보면서, Memory를 갖고 있고 다음 것을 하는 Neural Autoregressive Model같은 것 들을 보면, 
    - Conditional RNN(전체(?))를 environment로 보고, NN이 안에서 계산한 continuous vector를 세상의 state로 input으로 들어가는 것을 action, output을 observation으로 볼 수 있다.
    - 이러한 것을 활용해 기존의 Reinforcement Learning과 관련된 알고리즘을 모두 다 쓸 수 있다. (잘된다는 것은 아니다. 대부분의 Model free RL들이 잘 안되어서)
    - 다음과 같이
    - <img  src="./image/img_c4-9.PNG" width="80%">
    - 이를 통해, Autoregressive 모델이 무얼하고, 어떻게 행동을 할지 control할 수 있다.
    - 시행착오를 통해 할 수 있다는 것을 알았다. 
        - Decoder 를 에이전트(agent) 로 보고 훈련 가능한 Decoding agent 를 쌓는다.
        

### Simultaneous Translation

- 이 방법을 Simultaneous Translation(동시 통역)에 적용해 보았다.
    - 문장이 통째로 주어지는 것이 아니라 token 하나씩 주어질 때, 들어올 때마다 translate도 하나씩 generate가능 하게 하여
    - 문제를 다음 source token을 기다릴 것인지, 지금 와있는 것으로 translate을 generate 할 것인지로
    - 그리고 Objective는 quality는 높고 delay(source token이 들어오는 시점과 translation이 끝나는 시점의 차이를 minimize)는 적게하게
    
- <img  src="./image/img_c4-10.PNG" width="70%">
    - 노란색 색 부분은 training을 시켜 놓고 끝난 다음에, 그 위에 RL의 Agent를 train한다.
    - Agent는 매 time step 마다 NN안에 무엇이 일어나는지 (Decoder상의 Memory vector, Encoder상의 time-dependent Context vector를) 보고 source token을 기다릴 것인지, 지금 와있는 것으로 translate을 generate 할 것인지 결정한다.
    - RL은 Supervised과 다르게 Trial & Error style의 데이터를 collect하면서 Agent를 train하는 learning이니까 처음에는 막하다가 잘된 애를 Reinforce한다.
    - Policy gradient의 가장 단순한 Reinforce를 사용했고
    - Reward는 Trade-off between delay and quality로 설정했다.
    
    
- train은 생각보다 간단하게 되었다. GER $ \to $ ENG의 Attention Visualization
- <img  src="./image/img_c4-11.PNG" width="100%">
    - (b)는 기존의 translation model
    - 왼쪽을 보면 "The" 부터 "Being" 전까지 evidence를 기다렸다가 어느 정도 모이면 바로 translate을 시작한 것을 볼 수 있다.
    
    
    
- 여러 Language에 관한 결과.
    - <img  src="./image/img_c4-12.PNG" width="80%">
    - Average proportion이 Delay
    - 검정별이 기존의 translation, 잘되고 시간도 오래걸림
    - 파란색은 word by word translation: word by word라 당연히 잘안됨 시간은 빨라도
    - 하늘색 영역이 교수님이 혼자서 만들어본 decoding 알고리즘들
        - 잘 안됬다.
    - 보라색 영역이 어느 정도 Desired의 방향으로 끌어 올린 것을 볼 수 있다.
    

- 마무리: 이렇게 ML 문제를 풀다보면 지금까지 계속 봐왔듯이 문제가 한 두군데 있는 것이아니다. 
    - 어떤 Loss Function을 쓰고, 그에 따른 optimization을 쓰고(새로 만들어야 할 수도) train을 잘했다 하더라도
    - 또 train이 되어서 끝이 아니라 output이 복잡하면 train이 된 모델을 어떻게 쓸 것인지(Inference algorithm)
    - Inference algorithm 자체도 연구의 주제이고, 여기에 위에 보시다 시피 learning을 넣을 수도 있고
    - 또한 만약 모델을 Multi-task나 transfer learning으로 간다 하면은 다시 train할 때 그냥 optimization이 아니라 optimization을 어떻게 잘할 수 있는지 optimize해야하는 문제 

- 그럼에도 NLP에 딥러닝을 사용하면 좋은 점이 굉장히 compact한 set of tool을 써서 여러 문제에 쓸 수 있다는 것을 볼 수있다. 특히 case1~4 . 실제로 코드가 거의 같다.

- 지금까지 배운 principle을 잘 기억하면 다양한 문제를 푸는데 도움이 될 것 같다.
    - ** 끝 **

    
    
## CaseStudy 1-4 Q&A

- Q. 만원을 번역할 때, 10000 won, ten thousnad won 2가지로 번역이 될 수있는데 이때 가중치가 같다면 어떤 것이 합리적인지?
    - Context에 따라가야한다. 숫자, 연도, 사람이름, 지역이름과 같은 경우에서 production을 만들 때에는
    - 숫자 토큰으로 만들어 나중에 translate할 때 바꿔치기 한다든지, 사람 이름과 같은 것도 따로 engine을 써서 바꿔치기를 한다.
    - 지금까지 배운 것들은 어떻게 보면 Researcher들이 볼만한 것들이다.
    - production에는 해결해야 할 것들이 너무 많다.
        - 얼마전 페북의 번역 실수로 이스라엘과 팔레스타인에 문제가 있었다.
    - production을 위해서 post-processing, pre-processing, clean-up 코드들이 어마어마하게 많이 있다. 
    - 교수님은 연구자셔서 잘 모르고 AMTA(Association of Machine Translation in America)와 같은 학회에서는 production문제를 많이 다루므로 이쪽 논문을 보는 것이 많이 도움이 될 것이다.


- Q. Character level 임베딩 차원은 어떤 크기로 정해야 하는지?
    - A. 원래는 작아도 된는데, Default로 비교해야해서 300d로 했다.
    - Q. 일단 해봐야 하는 건가?
    - A. 이런 것들 전체가 다 하이퍼파라미터이다. 지금 모델들도 훨씬 크게 하면 더 잘될 것 같은데 컴퓨터 자원을 낮추려다 보니 못하는 것이다.
    - 추천하는 것은 GPU메모리가 허락하는 선에서 최대한 크게 이다.
    
    
- Q. 단어에 “성별” 의미가 없는 언어에서 내포된 언어로 번역할때 발생하는 문제점은 어떻게 해결하는지? ex) he, she
    - A. 2~3가지 해결방법이 있을 것이다. 지금까지 translation 시스템들이 너무 context없이 문장(source문장(?))만 본다.
    - 예를들어 대통령을 불어로 바꾸면 성별을 정해줘야하는데 이것이 NN입장에서는 정답을 갖고있지 않는 것이다. 
    - 한 가지 방법은 NN에 Context 정보까지 주는 것.(말한사람이 누군지, 듣는 사람이 누군지.)
        - 그러나 context 정보까지 줘도 데이터셋 내에 바이아이스가 있기 때문에, distribution을 learning하면은 데이터셋 내에서 얼마나 어떤 정보라던지 특정 pronoun이 prefer 됫냐에 따라서 Decoding이 완벽하지 않아 잘 안될 수도 있다.
    - 두 번째 방법은 따로 넣기전에(?) explicit하게 filtering이나 cleaning을 통해 해결
        - 근데 이게 되면 모델도 원래 할 줄 알아야 하는 것 같아 연구가 많이 필요
    - 그리고 마지막으로는 어쩌면 모델이 이미 learning을 잘 하고 있는데 decoding을 잘 못하는 것일 수도 있다.
    - 결국 모두 research topic이다.
    


- Q. Universal linguistic semantic space가 존재한다고 생각하는지? 한국어는 안되는데
    - A. 없다고 생각하지만, 인도유로피안이나 유로피안기반끼리는 잘됬었고,
    - 원래 한국어는 거의 연관있는 언어가 없어(나머지는 거의 비슷한 부분이 있는데) 아직 존재하지 않다고 단정짓기는 어렵다.
    


- Q. Decoding에서 완벽한 solution이 없다고 했으니 Autoregressive하게 결과를 내고, 그 결과를 Refine하는 더 좋은 것을 만드는 연구나 시도가 없는지.
    - A. MT가 실제로 실전에서 쓰일 때는 전문가들이 MT의 결과를 보고 post editting한다.
    - 그래서 이 Post editting을 자동으로 하는 것은 많이 연구하고 있다.
    - 교수님이 최근에 하고 있는 연구는 Non-오토리그레시브하게 하되 대신에 한번 translation을 하면 계속 refine하는 식으로 해보고 있다.
        - 그래도 sentence자체의 dependency가 너무 쎄서 아직까지 Autoregressive하게 하는 시스템 보다 좋은 것은 없는 것 같다.




<br>
<br>

### Reference: 

- https://www.edwith.org/deepnlp 조경현교수님, 딥러닝을 이용한 자연어 처리 강의 및 강의 자료 

### Papers:

- https://www.edwith.org/deepnlp/lecture/29227/
